I am a begginer of kaggle and learned a lot from Luke M's notebook. 
https://www.kaggle.com/deepdivelm/feature-engineering-lightgbm-exploring-performance

The score is surprisingly good, I share this notebook.

In [1]:
import pandas as pd
import numpy as np

In [1]:
#From Luke M's notebook
#https://www.kaggle.com/deepdivelm/feature-engineering-lightgbm-exploring-performance
df = pd.read_pickle('../input/df_complete.pkl')

In [1]:
#PS4 first sale item shuld be flagged
df['new_item_for_PS4'] = 0
df.loc[(df['category_id']==20) & (df['prev_days_on_sale']==0), 'new_item_for_PS4'] = 1

In [1]:
#Dec's sale RMSE is large, so I remove it.
df= df[df['month']!=12]

Model Learned from the above Luke M's notebook

In [1]:
import lightgbm as lgb

pd.set_option('display.max_rows', 160)
pd.set_option('display.max_columns', 160)
pd.set_option('display.max_colwidth', 30)

import warnings
warnings.filterwarnings("ignore")

In [1]:
X_train = df[~df.date_block_num.isin([0,1,33,34])]
y_train = X_train['item_cnt']
del X_train['item_cnt']

X_val = df[df['date_block_num']==33]
y_val = X_val['item_cnt']
del X_val['item_cnt']

X_test = df[df['date_block_num']==34].drop(columns='item_cnt')
X_test = X_test.reset_index()
del X_test['index']

#free memory
del df

In [1]:
def build_lgb_model(params, X_train, X_val, y_train, y_val, cat_features):
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_val = lgb.Dataset(X_val, y_val)
    model = lgb.train(params=params, train_set=lgb_train, valid_sets=(lgb_train, lgb_val), verbose_eval=50,
                     categorical_feature=cat_features)
    return model

In [1]:
params = {
    'objective': 'rmse',
    'metric': 'rmse',
    'num_leaves': 1023,
    'min_data_in_leaf':10,
    'feature_fraction':0.7,
    'learning_rate': 0.01,
    'num_rounds': 1000,
    'early_stopping_rounds': 30,
    'seed': 1
}
#designating the categorical features which should be focused on
cat_features = ['category_id','month','shop_id','shop_city']

lgb_model = build_lgb_model(params, X_train, X_val, y_train, y_val, cat_features)

#save model for later use
lgb_model.save_model('lgb_model.txt')

In [1]:
submission = pd.read_csv('../input/sample_submission.csv')
submission['item_cnt_month'] = lgb_model.predict(X_test).clip(0,20)

I guess that new release items in 2015 Nov such as Fallout4, battlefront, and one direction music should be predicred more sales, because the world sales of them is so large. To investigate the hypothesis, I multiplied the items' sales by 1.75. 

In [1]:
submission_old = submission
test = pd.read_csv('../input/test.csv')
submission_old = pd.merge(submission_old, test, on=['ID'], how='left')

In [1]:
#battlefront
submission_old.loc[submission_old['item_id'].isin(range(6729, 6733)), 'item_cnt_month'] *= 1.75
#fallout4
submission_old.loc[submission_old['item_id'].isin(range(3405, 3409)), 'item_cnt_month'] *= 1.75
#one direction; Made in the A.M.
submission_old.loc[submission_old['item_id'].isin(range(5320, 5323)), 'item_cnt_month'] *= 1.75

In [1]:
submission_old[['ID', 'item_cnt_month']].to_csv('lgb_submission.csv', index=False)

The hypothesis seems right because the score was imporved. I tried to applied the feature to the model, but I couldn't...